In [1]:
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, Dense, LSTM
import numpy as np
import string
import pickle

#### Load beer and brahma references to train the Model

In [2]:
def clean_sentence(sentence):
    sentence = sentence.replace(",", "")
    sentence = sentence.replace(".", "")
    sentence = sentence.replace("-", "")
    sentence = sentence.replace("/", "")
    sentence = sentence.replace(":", "")
    sentence = sentence.replace("!", "")
    sentence = sentence.replace("\"", "")
    sentence = sentence.replace("“","")
    return sentence.lower()

corpus = ""
with open('todas_campanhas.txt', 'r') as f:
    datastore = json.load(f)
    for item in datastore:
            corpus += " " + clean_sentence(item['titulo'])

with open('frases_famosos.txt', 'r') as f:
    datastore = json.load(f)
    for item in datastore:
            corpus += " " + clean_sentence(item['frase'])

            
#print(corpus)

#### Create sentences for each word in the corpus using the next "window_size" words

In [3]:
window_size=10
max_sequence_len = 30
sentences = []
alltext = []
words = corpus.split(" ")
print("Number words: ", len(words))
range_size = len(words) - max_sequence_len
for i in range(0, range_size):
    thissentence = ""
    for word in range(0, window_size-1):
        word = words[i+word]
        thissentence = thissentence + word
        thissentence = thissentence + " "
    sentences.append(thissentence)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
total_words = len(tokenizer.word_index) + 1
print("Number of unique words: ", total_words)
print("Number of sentences generated: ", len(sentences))

Number words:  1474
Number of unique words:  594
Number of sentences generated:  1444


#### Tokenize text sequences, generate feature and labels for training


In [4]:
input_sequences = []
for line in sentences:
    #print(line)
    token_list = tokenizer.texts_to_sequences([line])[0]
    #print(token_list)
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

xs, labels = input_sequences[:,:-1], input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)


#### Creating the Model

In [5]:
model = Sequential()
model.add(Embedding(total_words, 4))
model.add(Bidirectional(LSTM(max_sequence_len-1, return_sequences='True')))
model.add(Bidirectional(LSTM(max_sequence_len-1)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


#### Training the Model

In [6]:
history = model.fit(xs, ys, epochs=500, verbose=1)
print(model.summary())

Epoch 1/10
361/361 [==============================] - 12s 15ms/step - loss: 5.8570 - accuracy: 0.0591
Epoch 2/10
361/361 [==============================] - 6s 16ms/step - loss: 5.4421 - accuracy: 0.0642
Epoch 3/10
361/361 [==============================] - 6s 17ms/step - loss: 5.0565 - accuracy: 0.0737
Epoch 4/10
361/361 [==============================] - 7s 21ms/step - loss: 4.8453 - accuracy: 0.0749
Epoch 5/10
361/361 [==============================] - 8s 22ms/step - loss: 4.6782 - accuracy: 0.0824
Epoch 6/10
361/361 [==============================] - 9s 24ms/step - loss: 4.5819 - accuracy: 0.0841
Epoch 7/10
361/361 [==============================] - 8s 23ms/step - loss: 4.4506 - accuracy: 0.0903
Epoch 8/10
361/361 [==============================] - 9s 24ms/step - loss: 4.3607 - accuracy: 0.1039
Epoch 9/10
361/361 [==============================] - 9s 24ms/step - loss: 4.2552 - accuracy: 0.1037
Epoch 10/10
361/361 [==============================] - 9s 25ms/step - loss: 4.2358 - accur

#### Function to generate the sentence
seed_text: input text that will be used to generate the next word

next_words: number of words to be generated

number_options: number of top ranked words by the ML

In [7]:
def generate_sentence(seed_text, next_words, number_options=1):
    _seed_text = seed_text
    if next_words == 0:
        print(seed_text)
        return
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([_seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        arr = model.predict(token_list)
        ind = np.argpartition(np.array(arr[0]), -number_options)[-number_options:]
        output_word = ''

        for word, index in tokenizer.word_index.items():
            if index in ind:
                output_word = word
                seed_text_temp = _seed_text
                seed_text_temp += " " + output_word
                generate_sentence(seed_text_temp, (next_words - 1), number_options)
        break;

#### Example to print a sentence with 9 next words and just the first top ranked word

In [8]:
print(generate_sentence("cerveja", 9))

cerveja a cerveja a cerveja a cerveja a cerveja a
None


#### Example to print a sentence with 5 next words and 3 top ranked words

In [9]:
print(generate_sentence("brindar com brahma é", 5, 3))

brindar com brahma é cerveja a o brasil brahma
brindar com brahma é cerveja a o brasil de
brindar com brahma é cerveja a o brasil do
brindar com brahma é cerveja a o chopp brahma
brindar com brahma é cerveja a o chopp e
brindar com brahma é cerveja a o chopp da
brindar com brahma é cerveja a o lança brahma
brindar com brahma é cerveja a o lança se
brindar com brahma é cerveja a o lança não
brindar com brahma é cerveja a cerveja a o
brindar com brahma é cerveja a cerveja a cerveja
brindar com brahma é cerveja a cerveja a copa
brindar com brahma é cerveja a cerveja o brasil
brindar com brahma é cerveja a cerveja o chopp
brindar com brahma é cerveja a cerveja o lança
brindar com brahma é cerveja a cerveja que cerveja
brindar com brahma é cerveja a cerveja que campanha
brindar com brahma é cerveja a cerveja que você
brindar com brahma é cerveja a que cerveja cerveja
brindar com brahma é cerveja a que cerveja que
brindar com brahma é cerveja a que cerveja um
brindar com brahma é cerveja a q

brindar com brahma é ao a cerveja que cerveja
brindar com brahma é ao a cerveja que campanha
brindar com brahma é ao a cerveja que você
brindar com brahma é ao a copa brahma brasil
brindar com brahma é ao a copa brahma lança
brindar com brahma é ao a copa brahma estreia
brindar com brahma é ao a copa de cerveja
brindar com brahma é ao a copa de mundo
brindar com brahma é ao a copa de ação
brindar com brahma é ao a copa do cerveja
brindar com brahma é ao a copa do mundo
brindar com brahma é ao a copa do ação
brindar com brahma é ao o brasil brahma brasil
brindar com brahma é ao o brasil brahma lança
brindar com brahma é ao o brasil brahma estreia
brindar com brahma é ao o brasil de cerveja
brindar com brahma é ao o brasil de brasil
brindar com brahma é ao o brasil de leva
brindar com brahma é ao o brasil do cerveja
brindar com brahma é ao o brasil do mundo
brindar com brahma é ao o brasil do ação
brindar com brahma é ao o chopp brahma chopp
brindar com brahma é ao o chopp brahma cria
br